# AI Hub 전사 규칙 처리
- Reference: `https://blog.naver.com/PostView.nhn?blogId=sooftware&logNo=221821797852`

In [4]:
test1 = "o/ 근데 (70%)/(칠십 퍼센트)가 커 보이긴 하는데 (200)/(이백) 벌다 (140)/(백 사십) 벌면 빡셀걸? b/"
test2 = "근데 (3학년)/(삼 학년) 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?"

# Whisper 전처리는 Reference와 다르므로 약간의 수정이 필요하다.
def bracket_filter(sentence):
    new_sentence = str()
    """
    (left_bracket, right_bracket)
    (0, 0) -> 출력
    (1, 0) -> 출력
    (1, 1) -> 미출력
    (0, 1) -> 미출력
    --> right_bracket을 기준으로 flag 설정
    """
    flag = False
    
    for ch in sentence:
        if ch == ')' and flag == False:
            flag = True
            continue
        if ch == ')' and flag == True:
            flag = False
            continue
        if ch != '(' and flag == False:
            new_sentence += ch
    return new_sentence

test1 = bracket_filter(test1)
test2 = bracket_filter(test2)
print(test1)
print(test2)

o/ 근데 70%가 커 보이긴 하는데 200 벌다 140 벌면 빡셀걸? b/
근데 3학년 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?


In [5]:
import re

def special_filter(sentence):
    SENTENCE_MASK = ['.', '?', ',', '!']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '~', ':', ';']
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MASK:
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/':
                continue
        if ch == '#':
            new_sentence += '샵'
        elif ch not in EXCEPT:
            new_sentence += ch

    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence

print(special_filter(test1))
print(special_filter(test2))

근데 70%가 커 보이긴 하는데 200 벌다 140 벌면 빡셀걸?
근데 3학년 때 까지는 국가장학금 바 받으면서 다녔던 건가?


In [9]:
import os
import json
import librosa
import soundfile as sf

def is_json(filename):
    return filename[-4:] == 'json'

# Whisper 모델의 전사 규칙에 알맞게 변환
def transcription(root_dir):
    len_root_dir = len(root_dir)
    for dirpath, dirnames, filenames in os.walk(root_dir):
        print("Processing:", dirpath)
        for filename in filenames:
            if is_json(filename):
                filepath = os.path.join(dirpath, filename)
                with open(filepath, "r", encoding="utf-8") as json_file:
                    data = json.load(json_file)
                    try:
                        text = data["annotations"][0]["note"]
                    except:
                        text = data["annotations"]["note"]

                    # 전사 규칙 처리
                    text = bracket_filter(text)
                    text = special_filter(text)
                    
                    new_data = {
                        "annotations" : {
                            "note" : text
                        }
                    }
                    
                    with open(filepath, 'w', encoding='utf-8') as file:
                        json.dump(new_data, file, ensure_ascii=False)

    print("Done!")

In [10]:
root_directory = "D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset"
transcription(root_directory)

Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\1.강제추행(성범죄)
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\10.낙상
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\11.붕괴사고
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\12.태풍-강풍
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\13.지진
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\14.도움요청
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\15.실내
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\16.실외
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\17.가전소음_세탁기,건조기
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\test\audio\18.가전소음_청소기
Processing: D:/01 서울시립대학교/03 5학년 2학기/01 소프트웨어응용/Dataset\t